In [4]:
import cv2 as cv
import numpy as np
import tensorflow as tf

from helpers.utils import show_image, normalize, preprocess_image, detect_sudoku, extract_cells

In [5]:
def extract_information(img):
    model = tf.keras.models.load_model('saved_model/model.h5')
    img = detect_sudoku(img)  # varying size, colored
    img = cv.resize(img, (500, 500))  # fixed size
    # show_image("sudoku", img)  # show cropped image (sudoku rectangle)

    # apply lines overlay:
    # for line in lines_vertical:
    #     cv.line(img, line[0], line[1], (0, 255, 0), 5)
    # for line in lines_horizontal:
    #     cv.line(img, line[0], line[1], (0, 0, 255), 5)
    # show_image("img", img)  # show sudoku with lines overlay

    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)  # fixed size, grayscale

    cells = extract_cells(img)  # each cell is a 28x28 grayscale image
    cells = cells.reshape((cells.shape[0], 28, 28, 1)).astype('float32')
    predictions = model.predict(cells)  # make predictions
    labels = [np.argmax(prediction) for prediction in predictions]  # extract labels
    return labels



In [6]:
def get_results(input_dir, output_dir, number_of_samples):
    for i in range(1, number_of_samples + 1):
        if i < 10:
            img = cv.imread(
                f"{input_dir}/0{i}.jpg")
        else:
            img = cv.imread(
                f"{input_dir}/{i}.jpg") 
        cell_labels = np.array(extract_information(img))
        cell_labels = np.reshape(cell_labels, (9, 9))

        file = open(f'{output_dir}/clasic/{i}_predicted.txt', 'w')
        for j in range(9):
            for k in range(9):
                if cell_labels[j][k] == 0:
                    char = 'o'
                else:
                    char = 'x'
                file.write(char)
            if j != 8:
                file.write('\n')
        file.close()

        file = open(f'{output_dir}/clasic/{i}_bonus_predicted.txt', 'w')
        for j in range(9):
            for k in range(9):
                char = str(cell_labels[j][k])
                if char == '0':
                    char = 'o'
                file.write(char)
            if j != 8:
                file.write('\n')
        file.close()

In [7]:
input_dir = 'datasets/antrenare/clasic'
output_dir = 'results'
number_of_samples = 20  # number of input images
get_results(input_dir, output_dir, number_of_samples)

2021-11-29 16:21:18.792427: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-29 16:21:18.803692: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2021-11-29 16:21:18.803714: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-29 16:21:18.804732: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN